In [17]:
fp = r"C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\spec_example\spec2006_470.lbm\lbm_part.asm"

In [18]:
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 00000000000005e8
.plt 0000000000000600
__cxa_finalize@plt 0000000000000620
__libc_start_main@plt 0000000000000630
__gmon_start__@plt 0000000000000640
abort@plt 0000000000000650
printf@plt 0000000000000660
_start 0000000000000670
call_weak_fn 00000000000006a8
deregister_tm_clones 00000000000006c0
register_tm_clones 00000000000006f0
__do_global_dtors_aux 0000000000000730
frame_dummy 0000000000000778
main 000000000000077c
initializerandommesh 00000000000007dc
swapMesh 000000000000086c
SET_FLAG 00000000000008ac
__libc_csu_init 0000000000000920
__libc_csu_fini 00000000000009a0
_fini 00000000000009a4


In [19]:
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output\\procedures.svg'

In [20]:
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output\\call_graph.svg'

In [21]:
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
for l in tcfg.loops:
    l.back_edge.is_backEdge = True
for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)


#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




n0 n3 1.0
n13 n1 0.5
n1 n14 0.5
n14 n2 0.5
n3 n12 1.0
n4 n10 0.5
n5 n8 0.25
n6 n15 0.125
n15 n7 0.125
n7 n8 2
n8 n6 0.125
n8 n9 0.125
n9 n10 2
n10 n5 0.25
n10 n11 0.25
n11 n12 2
n12 n4 0.5
n12 n13 0.5


In [22]:
from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table

for i in lds_table:
    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr)

rwproc = RWProc(lds_table)

#for rwu in rwproc.rw_table:
#    print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)

指令 ('784', 'b9001fe0', 'str', None, 'w0, [sp, #28]') 是否是SP True 指令地址 1924 访存地址 0
指令 ('788', 'f9000be1', 'str', None, 'x1, [sp, #16]') 是否是SP True 指令地址 1928 访存地址 0
指令 ('790', 'f947fc00', 'ldr', None, 'x0, [x0, #4088]') 是否是SP False 指令地址 1936 访存地址 69624
指令 ('798', 'fd000000', 'str', None, 'd0, [x0]') 是否是SP False 指令地址 1944 访存地址 65536
指令 ('7a0', 'f947fc00', 'ldr', None, 'x0, [x0, #4088]') 是否是SP False 指令地址 1952 访存地址 69624
指令 ('7a4', 'f900041f', 'str', None, 'xzr, [x0, #8]') 是否是SP False 指令地址 1956 访存地址 65544
指令 ('7ac', 'f947fc00', 'ldr', None, 'x0, [x0, #4088]') 是否是SP False 指令地址 1964 访存地址 69624
指令 ('7b8', 'f947fc00', 'ldr', None, 'x0, [x0, #4088]') 是否是SP False 指令地址 1976 访存地址 69624
指令 ('7c0', 'fd6f4000', 'ldr', None, 'd0, [x0, #24192]') 是否是SP False 指令地址 1984 访存地址 89976
指令 ('7e4', 'f9000fe0', 'str', None, 'x0, [sp, #24]') 是否是SP True 指令地址 2020 访存地址 0
指令 ('7e8', 'b9002fff', 'str', None, 'wzr, [sp, #44]') 是否是SP True 指令地址 2024 访存地址 0
指令 ('7f0', 'b9002bff', 'str', None, 'wzr, [sp, #40]') 是否是SP True 指令

In [23]:
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True


for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))

l2 (in l1 children []): n7->n8 ['n6', 'n15', 'n7', 'n8']
l1 (in l0 children ['l2']): n9->n10 ['n9', 'n6', 'n15', 'n5', 'n7', 'n8', 'n10']
l0 (in None children ['l1']): n11->n12 ['n9', 'n6', 'n15', 'n5', 'n4', 'n12', 'n7', 'n8', 'n10', 'n11']
n0 None
n1 None
n2 None
n3 None
n4 l0
n5 l1
n6 l2
n7 l2
n8 l2
n9 l1
n10 l1
n11 l0
n12 l0
n13 None
n14 None
n15 l2
n16 None
